In [5]:
import cv2


step=1
for i in range (1 ,1100,step) :
    I = cv2 . imread ( "pedestrian/pedestrian/input/in%06d.jpg" % i )
    cv2.imshow ("I" ,I )
    cv2.waitKey (10)
cv2.destroyAllWindows()

In [20]:
import cv2
import numpy as np

step = 1
kernel = np.ones((3,3), np.uint8)  # Jądro do operacji morfologicznych

for i in range(300, 1100, step):
    # Wczytaj obraz
    I = cv2.imread("pedestrian/pedestrian/input/in%06d.jpg" % i)



    # Konwersja na obraz w skali szarości
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    (T , thresh ) = cv2.threshold (gray ,130 ,255 , cv2.THRESH_BINARY )
    # Dylatacja
    dilated = cv2.dilate(gray, kernel, iterations=1)

    # Erozja
    eroded = cv2.erode(gray, kernel, iterations=1)

    abdsdiff=cv2.absdiff(eroded, dilated)

    # Wyświetlenie oryginału, dylatacji i erozji
    cv2.imshow("Original", gray)
    cv2.imshow("Dilated", dilated)
    cv2.imshow("Eroded", eroded)
    cv2.imshow("Diff eroded and dilated",abdsdiff)
    cv2.imshow("Binarization",thresh)

    # Czekaj na chwilę (10 ms) przed przejściem do następnego obrazu
    cv2.waitKey(10)

cv2.destroyAllWindows()


Piękne 92% F1Score dla pedestrians

In [21]:
import cv2
import numpy as np
import os

step = 1
output_folder = "pedestrian/pedestrian/output_masks"
os.makedirs(output_folder, exist_ok=True)
kernel = np.ones((5,5), np.uint8)  # szczerze nie wiem, czy 5,5 czy 7,7 jest lepszy
kernel_dil=np.ones((3,3),np.uint8)

# Inicjalizacja liczników
TP, TN, FP, FN = 0, 0, 0, 0

# Parametry detekcji ludzi
MIN_AREA = 600   # Minimalna powierzchnia obiektu (ignoruje szum)
MAX_AREA = 10000  # Maksymalna powierzchnia obiektu (pomija np. samochody)
RATIO_THRESHOLD = 1.1  # Minimalny stosunek wysokości do szerokości dla ludzi (raczej nie potrzebne, w tym przypadku)

# Zmienna przechowująca tło dla detekcji ruchu
avg_frame = None  

for i in range(300, 1100, step):
    # Wczytaj obraz
    I = cv2.imread("pedestrian/pedestrian/input/in%06d.jpg" % i)

    # Konwersja do skali szarości
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    # Inicjalizacja tła dla detekcji ruchu (inicjalizacja tła dla pierwszej klatki)
    if avg_frame is None:
        avg_frame = np.float32(gray)
    
    # Aktualizacja średniej tła (to jest po to, ponieważ jak po prostu sprawdzało klatka po klatce ruch to nie wykrywało kogoś kto wolno chodzi, a tak to zbiera średnią z kilku klatek)
    cv2.accumulateWeighted(gray, avg_frame, 0.005) #musi byc w float32 bo działa lepiej
    background = cv2.convertScaleAbs(avg_frame)#powró do uint8

    # Detekcja ruchu (różnica między aktualną klatką a tłem)
    motion_mask = cv2.absdiff(gray, background)
    _, motion_mask = cv2.threshold(motion_mask, 43, 255, cv2.THRESH_BINARY)
    motion_mask = cv2.dilate(motion_mask, kernel_dil, iterations=1)  # Powiększenie obszaru ruchu

    # Adaptacyjne progowanie
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 2)#lepszy niż otsu i ręcznie dobierany próg

    # Operacja zamknięcia (zamyka przerwy w konturach ludzi)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2) #to jest dilatate i erosion w jednym

    #edges = cv2.Canny(gray, 50, 150)
    #morph = cv2.bitwise_and(morph, edges)


    # Połączenie maski ruchu i progowania
    combined_mask = cv2.bitwise_and(morph, motion_mask)
    

    #combined_mask =cv2.erode(combined_mask,kernel_dil,iterations=0)
    combined_mask = cv2.dilate(combined_mask, kernel_dil, iterations=1)  # Pełniejsze sylwetki

    combined_mask = cv2.medianBlur(combined_mask, 3)  # Redukuje szumy, stabilizuje maskę
    mask_filename = os.path.join(output_folder, f"mask_{i:06d}.png")
    cv2.imwrite(mask_filename, combined_mask)

    # Znajdowanie składowych połączonych
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(combined_mask, connectivity=8) #8-sąsiedztwo

    I_VIS = I.copy()

    # Wyrysowanie tylko obiektów przypominających ludzi
    for pi in range(1, stats.shape[0]):  # Zaczynamy od 1, bo indeks 0 to tło
        x, y, w, h, area = stats[pi]

        # Filtrujemy obiekty na podstawie powierzchni i kształtu
        if MIN_AREA < area < MAX_AREA and h > w * RATIO_THRESHOLD:
            cv2.rectangle(I_VIS, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(I_VIS, f"ID: {pi}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            cv2.putText(I_VIS, f"{area} px", (x, y + h + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    

    # Wczytanie maski referencyjnej (groundtruth)
    GTB = cv2.imread(f"pedestrian/pedestrian/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE)

    if GTB is None:
        print(f"Brak groundtruth dla klatki {i}")
        continue  # Pomijamy brakujące klatki

    # Konwersja groundtruth do binarnej postaci (upewniamy się, że tylko 0 i 255)
    GTB[GTB > 0] = 255  

    # Obliczenie metryk przy użyciu operacji macierzowych
    TP += np.sum(np.logical_and((combined_mask == 255), (GTB == 255)))  # Prawdziwe pozytywy
    FP += np.sum(np.logical_and((combined_mask == 255), (GTB == 0)))    # Fałszywe pozytywy
    FN += np.sum(np.logical_and((combined_mask == 0), (GTB == 255)))    # Fałszywe negatywy
    TN += np.sum(np.logical_and((combined_mask == 0), (GTB == 0)))      # Prawdziwe negatywy

    good_diff=cv2.absdiff(GTB,combined_mask)

    # Wyświetlanie przetworzonego obrazu
    cv2.imshow("Processed Image", I_VIS)
    cv2.imshow("Motion Mask",combined_mask)
    cv2.imshow("Difference between motion mask and gtb",good_diff)
    cv2.waitKey(10)

cv2.destroyAllWindows()

P = TP / (TP + FP) if (TP + FP) > 0 else 0  # Precision
R = TP / (TP + FN) if (TP + FN) > 0 else 0  # Recall
F1 = 2 * (P * R) / (P + R) if (P + R) > 0 else 0  # F1-score

# Wyświetlenie wyników
print(f"Precision: {P:.4f}")
print(f"Recall: {R:.4f}")
print(f"F1-score: {F1:.4f}")


Precision: 0.9077
Recall: 0.9339
F1-score: 0.9206


Highway

In [22]:
import cv2
import numpy as np
import os

step = 1
output_folder = "highway/highway/output_masks"
os.makedirs(output_folder, exist_ok=True)
kernel = np.ones((5,5), np.uint8)  # szczerze nie wiem, czy 5,5 czy 7,7 jest lepszy
kernel_dil=np.ones((3,3),np.uint8)

# Inicjalizacja liczników
TP, TN, FP, FN = 0, 0, 0, 0

# Parametry detekcji ludzi
MIN_AREA = 1000   # Minimalna powierzchnia obiektu (ignoruje szum)
MAX_AREA = 10000  # Maksymalna powierzchnia obiektu (pomija np. samochody)
RATIO_THRESHOLD = 0.3  # Minimalny stosunek wysokości do szerokości dla ludzi (raczej nie potrzebne, w tym przypadku)

# Zmienna przechowująca tło dla detekcji ruchu
avg_frame = None  

for i in range(470, 1700, step):
    # Wczytaj obraz
    I = cv2.imread("highway/highway/input/in%06d.jpg" % i)

    # Konwersja do skali szarości
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    # Inicjalizacja tła dla detekcji ruchu (inicjalizacja tła dla pierwszej klatki)
    if avg_frame is None:
        avg_frame = np.float32(gray)
    
    # Aktualizacja średniej tła (to jest po to, ponieważ jak po prostu sprawdzało klatka po klatce ruch to nie wykrywało kogoś kto wolno chodzi, a tak to zbiera średnią z kilku klatek)
    cv2.accumulateWeighted(gray, avg_frame, 0.01) #musi byc w float32 bo działa lepiej
    background = cv2.convertScaleAbs(avg_frame)#powró do uint8

    # Detekcja ruchu (różnica między aktualną klatką a tłem)
    motion_mask = cv2.absdiff(gray, background)
    _, motion_mask = cv2.threshold(motion_mask, 58, 255, cv2.THRESH_BINARY)
    motion_mask = cv2.dilate(motion_mask, kernel_dil, iterations=1)  # Powiększenie obszaru ruchu

    # Adaptacyjne progowanie
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 2)#lepszy niż otsu i ręcznie dobierany próg

    # Operacja zamknięcia (zamyka przerwy w konturach ludzi)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2) #to jest dilatate i erosion w jednym

    #edges = cv2.Canny(gray, 50, 150)
    #morph = cv2.bitwise_and(morph, edges)


    # Połączenie maski ruchu i progowania
    combined_mask = cv2.bitwise_and(morph, motion_mask)
    

    #combined_mask =cv2.erode(combined_mask,kernel_dil,iterations=0)
    combined_mask = cv2.dilate(combined_mask, kernel_dil, iterations=1)  # Pełniejsze sylwetki

    combined_mask = cv2.medianBlur(combined_mask, 3)  # Redukuje szumy, stabilizuje maskę
    mask_filename = os.path.join(output_folder, f"mask_{i:06d}.png")
    cv2.imwrite(mask_filename, combined_mask)

    # Znajdowanie składowych połączonych
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(combined_mask, connectivity=8) #8-sąsiedztwo

    I_VIS = I.copy()

    # Wyrysowanie tylko obiektów przypominających ludzi
    for pi in range(1, stats.shape[0]):  # Zaczynamy od 1, bo indeks 0 to tło
        x, y, w, h, area = stats[pi]

        # Filtrujemy obiekty na podstawie powierzchni i kształtu
        if MIN_AREA < area < MAX_AREA and h > w * RATIO_THRESHOLD:
            cv2.rectangle(I_VIS, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(I_VIS, f"ID: {pi}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            cv2.putText(I_VIS, f"{area} px", (x, y + h + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    

    # Wczytanie maski referencyjnej (groundtruth)
    GTB = cv2.imread(f"highway/highway/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE)

    if GTB is None:
        print(f"Brak groundtruth dla klatki {i}")
        continue  # Pomijamy brakujące klatki

    # Konwersja groundtruth do binarnej postaci (upewniamy się, że tylko 0 i 255)
    GTB[GTB > 0] = 255  

    # Obliczenie metryk przy użyciu operacji macierzowych
    TP += np.sum(np.logical_and((combined_mask == 255), (GTB == 255)))  # Prawdziwe pozytywy
    FP += np.sum(np.logical_and((combined_mask == 255), (GTB == 0)))    # Fałszywe pozytywy
    FN += np.sum(np.logical_and((combined_mask == 0), (GTB == 255)))    # Fałszywe negatywy
    TN += np.sum(np.logical_and((combined_mask == 0), (GTB == 0)))      # Prawdziwe negatywy

    good_diff=cv2.absdiff(GTB,combined_mask)

    # Wyświetlanie przetworzonego obrazu
    cv2.imshow("Processed Image", I_VIS)
    cv2.imshow("Motion Mask",combined_mask)
    cv2.imshow("Difference between motion mask and gtb",good_diff)
    cv2.waitKey(10)

cv2.destroyAllWindows()

P = TP / (TP + FP) if (TP + FP) > 0 else 0  # Precision
R = TP / (TP + FN) if (TP + FN) > 0 else 0  # Recall
F1 = 2 * (P * R) / (P + R) if (P + R) > 0 else 0  # F1-score

# Wyświetlenie wyników
print(f"Precision: {P:.4f}")
print(f"Recall: {R:.4f}")
print(f"F1-score: {F1:.4f}")

Precision: 0.8898
Recall: 0.8779
F1-score: 0.8838


Office

In [17]:
import cv2
import numpy as np
import os

step = 15
output_folder = "office/office/output_masks"
os.makedirs(output_folder, exist_ok=True)
kernel = np.ones((5,5), np.uint8)  # szczerze nie wiem, czy 5,5 czy 7,7 jest lepszy
kernel_dil=np.ones((3,3),np.uint8)

# Inicjalizacja liczników
TP, TN, FP, FN = 0, 0, 0, 0

# Parametry detekcji ludzi
MIN_AREA = 5500   # Minimalna powierzchnia obiektu (ignoruje szum)
MAX_AREA = 12000  # Maksymalna powierzchnia obiektu (pomija np. samochody)
RATIO_THRESHOLD = 1.1  # Minimalny stosunek wysokości do szerokości dla ludzi (raczej nie potrzebne, w tym przypadku)

# Zmienna przechowująca tło dla detekcji ruchu
avg_frame = None  

for i in range(570, 2050, step):
    # Wczytaj obraz
    I = cv2.imread("office/office/input/in%06d.jpg" % i)

    # Konwersja do skali szarości
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    # Inicjalizacja tła dla detekcji ruchu (inicjalizacja tła dla pierwszej klatki)
    if avg_frame is None:
        avg_frame = np.float32(gray)
    
    # Aktualizacja średniej tła (to jest po to, ponieważ jak po prostu sprawdzało klatka po klatce ruch to nie wykrywało kogoś kto wolno chodzi, a tak to zbiera średnią z kilku klatek)
    cv2.accumulateWeighted(gray, avg_frame, 0.01) #musi byc w float32 bo działa lepiej
    background = cv2.convertScaleAbs(avg_frame)#powró do uint8

    # Detekcja ruchu (różnica między aktualną klatką a tłem)
    motion_mask = cv2.absdiff(gray, background)
    _, motion_mask = cv2.threshold(motion_mask, 45, 255, cv2.THRESH_BINARY)
    motion_mask = cv2.dilate(motion_mask, kernel_dil, iterations=1)  # Powiększenie obszaru ruchu

    # Adaptacyjne progowanie
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 2)#lepszy niż otsu i ręcznie dobierany próg

    # Operacja zamknięcia (zamyka przerwy w konturach ludzi)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2) #to jest dilatate i erosion w jednym

    #edges = cv2.Canny(gray, 50, 150)
    #morph = cv2.bitwise_and(morph, edges)


    # Połączenie maski ruchu i progowania
    combined_mask = cv2.bitwise_and(morph, motion_mask)
    

    #combined_mask =cv2.erode(combined_mask,kernel_dil,iterations=0)
    combined_mask = cv2.dilate(combined_mask, kernel_dil, iterations=1)  # Pełniejsze sylwetki

    combined_mask = cv2.medianBlur(combined_mask, 3)  # Redukuje szumy, stabilizuje maskę
    mask_filename = os.path.join(output_folder, f"mask_{i:06d}.png")
    cv2.imwrite(mask_filename, combined_mask)

    # Znajdowanie składowych połączonych
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(combined_mask, connectivity=8) #8-sąsiedztwo

    I_VIS = I.copy()

    # Wyrysowanie tylko obiektów przypominających ludzi
    for pi in range(1, stats.shape[0]):  # Zaczynamy od 1, bo indeks 0 to tło
        x, y, w, h, area = stats[pi]

        # Filtrujemy obiekty na podstawie powierzchni i kształtu
        if MIN_AREA < area < MAX_AREA and h > w * RATIO_THRESHOLD:
            cv2.rectangle(I_VIS, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(I_VIS, f"ID: {pi}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            cv2.putText(I_VIS, f"{area} px", (x, y + h + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    

    # Wczytanie maski referencyjnej (groundtruth)
    GTB = cv2.imread(f"office/office/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE)

    if GTB is None:
        print(f"Brak groundtruth dla klatki {i}")
        continue  # Pomijamy brakujące klatki

    # Konwersja groundtruth do binarnej postaci (upewniamy się, że tylko 0 i 255)
    GTB[GTB > 0] = 255  

    # Obliczenie metryk przy użyciu operacji macierzowych
    TP += np.sum(np.logical_and((combined_mask == 255), (GTB == 255)))  # Prawdziwe pozytywy
    FP += np.sum(np.logical_and((combined_mask == 255), (GTB == 0)))    # Fałszywe pozytywy
    FN += np.sum(np.logical_and((combined_mask == 0), (GTB == 255)))    # Fałszywe negatywy
    TN += np.sum(np.logical_and((combined_mask == 0), (GTB == 0)))      # Prawdziwe negatywy

    good_diff=cv2.absdiff(GTB,combined_mask)

    # Wyświetlanie przetworzonego obrazu
    cv2.imshow("Processed Image", I_VIS)
    cv2.imshow("Motion Mask",combined_mask)
    cv2.imshow("Difference between motion mask and gtb",good_diff)
    cv2.waitKey(10)

cv2.destroyAllWindows()

P = TP / (TP + FP) if (TP + FP) > 0 else 0  # Precision
R = TP / (TP + FN) if (TP + FN) > 0 else 0  # Recall
F1 = 2 * (P * R) / (P + R) if (P + R) > 0 else 0  # F1-score

# Wyświetlenie wyników
print(f"Precision: {P:.4f}")
print(f"Recall: {R:.4f}")
print(f"F1-score: {F1:.4f}")

Precision: 0.8346
Recall: 0.9017
F1-score: 0.8669
